<a href="https://colab.research.google.com/github/cuzmyk/data_analysis/blob/main/DA_2pr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическая работа №2. Применение классических методов машинного обучения в среде Apache Spark

### Необходимые зависимости (запустить)

In [1]:
!git clone --recursive https://github.com/tester170/Other.git

Cloning into 'Other'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 56 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 32.77 MiB | 10.08 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (26/26), done.


In [2]:
!ls Other/

archive.zip  beauty.csv  columns.csv  email.csv  images		 README.md	xl
bank.csv     births.csv  data.zip     excel.py	 passengers.csv  responses.csv	сollege_data.csv


In [3]:
!unzip "/content/Other/data.zip" -d "/content/"

Archive:  /content/Other/data.zip
   creating: /content/data/
  inflating: /content/__MACOSX/._data  
  inflating: /content/data/new_customers.csv  
  inflating: /content/__MACOSX/data/._new_customers.csv  
   creating: /content/data/ml-1m/
  inflating: /content/__MACOSX/data/._ml-1m  
  inflating: /content/data/Meal_Info.csv  
  inflating: /content/__MACOSX/data/._Meal_Info.csv  
  inflating: /content/data/fakefriends.csv  
  inflating: /content/__MACOSX/data/._fakefriends.csv  
  inflating: /content/data/sample_linear_regression_data.txt  
  inflating: /content/__MACOSX/data/._sample_linear_regression_data.txt  
  inflating: /content/data/College.csv  
  inflating: /content/__MACOSX/data/._College.csv  
  inflating: /content/data/fake_customers.csv  
  inflating: /content/__MACOSX/data/._fake_customers.csv  
  inflating: /content/data/movielens_ratings.csv  
  inflating: /content/__MACOSX/data/._movielens_ratings.csv  
  inflating: /content/data/Book.txt  
  inflating: /content/__MAC

In [4]:
!pip install pyspark
!pip install findspark

In [5]:
import findspark
findspark.init()

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('Practice2').getOrCreate()

## Задание №1. Линейная регрессия (Linear Regression)

### Построение прогностической модели для судов

[Hyundai Heavy Industries](http://www.hyundai.eu/en) - одна из ведущих судостроительных компаний в мире, специализирующаяся на строительстве круизных лайнеров. В настоящее время они работают над созданием новых кораблей для различных клиентов и стремятся к точному прогнозированию количества необходимых членов экипажа для каждого корабля.

Данные, с которыми мы работаем, представляют собой измерения размеров, вместимости, экипажа и возраста 158 круизных судов. Эти данные включают следующие столбцы:

- Ship_name
- Cruise_line
- Age (по состоянию на 2013 год)
- Tonnage (1000 тонн)
- Passengers (100 человек)
- Length (100 футов)
- Cabins (100 шт.)
- Passenger_density
- Crew (100s)

Эти данные хранятся в файле CSV под названием "cruise_ship_info.csv".

##### **Ваша задача - создать регрессионную модель, которая поможет предсказать, сколько членов экипажа потребуется для будущих кораблей.**

In [8]:
spark = SparkSession.builder.appName('Linear Regression').getOrCreate()
from pyspark.ml.regression import LinearRegression

In [9]:
df = spark.read.csv('data/cruise_ship_info.csv',inferSchema=True,header=True)

In [10]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [11]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [12]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       NULL|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     NULL|       NULL| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [13]:
# Прежде чем Spark сможет принять данные, нам нужно сделать несколько преобразований
# Данные должны быть в виде двух столбцов
# ("label", "features")
# Импортируем VectorAssembler и Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
# Создаем экземпляр класса VectorAssembler
assembler = VectorAssembler(
    # Указываем входные колонки, которые будут использоваться для создания вектора признаков
    inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'],
    # Указываем имя выходной колонки, в которой будет храниться вектор признаков
    outputCol="features")

In [15]:
# Применяем assembler к данным для создания вектора признаков
output = assembler.transform(df)

In [16]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [17]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [18]:
# Выбираем колонки 'features' и 'crew' из DataFrame 'output'

final_data = output.select("features",'crew')

In [19]:
# Разделяем данные на обучающий и тестовый наборы
# 70% данных идут в обучающий набор, 30% - в тестовый

train_data, test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.965175438596502|
| stddev|3.5583396328063244|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [21]:
# Создаём объект модели линейной регрессии
lr = LinearRegression(labelCol='crew')

In [22]:
# Обучим модель

lrModel = lr.fit(train_data,)

In [23]:
# Выводим коэффициенты и пересечение с осью Y (intercept) обученной модели

print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.009630042076769872,0.012926322156345824,-0.1892440012291379,0.3609478558887437,0.9170939969345575,0.0013321695636143683] Intercept: -0.5913880473774934


In [24]:
test_results = lrModel.evaluate(test_data)

In [25]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  1.0809007776323014|
| -0.3353662508486863|
|  0.5550049885440647|
|  0.6167243419039146|
| -0.1397543221418971|
| 0.27258883218757823|
|  0.9118075246093635|
| -0.4629607988798412|
| -0.6384086892108609|
|  1.7127870866566712|
| -0.3642082544012357|
| -0.4040114712464504|
|  0.7929773416273154|
| -1.3633191664740405|
|-0.03179551553337312|
|-0.06544946624309489|
|  0.7786443434224921|
| -1.3055599747349547|
|  0.8816235085503816|
|  0.0451210735147618|
+--------------------+
only showing top 20 rows



In [26]:
# Создаем немаркированный набор данных, выбирая только признаки из тестовых данных
unlabeled_data = test_data.select('features')

In [27]:
# Применяем модель к немаркированным данным для получения прогнозов
predictions = lrModel.transform(unlabeled_data)

In [28]:
# Выводим прогнозы
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...| 5.619099222367699|
|[6.0,30.276999999...| 3.885366250848686|
|[6.0,93.0,23.94,9...|10.534995011455935|
|[6.0,110.23899999...|10.883275658096085|
|[6.0,158.0,43.7,1...|13.739754322141897|
|[8.0,77.499,19.5,...| 8.727411167812422|
|[8.0,91.0,22.44,9...|10.088192475390636|
|[9.0,85.0,19.68,9...|  9.15296079887984|
|[9.0,110.0,29.74,...| 12.23840868921086|
|[10.0,46.0,7.0,6....|2.7572129133433285|
|[11.0,86.0,21.24,...| 9.664208254401236|
|[11.0,90.09,25.01...|  8.88401147124645|
|[11.0,108.977,26....|11.207022658372685|
|[11.0,138.0,31.14...| 13.21331916647404|
|[12.0,25.0,3.88,5...|2.9017955155333732|
|[12.0,50.0,7.0,7....| 4.515449466243095|
|[12.0,77.104,20.0...| 8.811355656577508|
|[12.0,88.5,21.24,...|10.605559974734955|
|[12.0,91.0,20.32,...| 9.108376491449619|
|[13.0,25.0,3.82,5...|2.9048789264852384|
+--------------------+------------

In [29]:
# Выводим корень из среднеквадратической ошибки (RMSE)
print("RMSE: {}".format(test_results.rootMeanSquaredError))

# Выводим среднеквадратическую ошибку (MSE)
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.6983950774972038
MSE: 0.4877556842723253


## Задание №2. Логистическая регрессия (Logistic Regression)

## Разработка модели бинарного классификатора для определения оттока клиентов

- Маркетинговое агентство имеет множество клиентов, которые используют их услуги для создания рекламы на веб-сайтах клиентов. Они заметили, что у них довольно большой отток клиентов. Необходимо создать модель машинного обучения, которая поможет предсказать, какие клиенты уйдут (перестанут покупать их услуги), чтобы они могли правильно назначить менеджера по работе с клиентами, которые наиболее вероятно уйдут.

- **Разработайте алгоритм классификации, который поможет определить, уйдет клиент или нет.** Затем компания может проверить это на входящих данных для будущих клиентов, чтобы предсказать, какие клиенты уйдут и назначить им менеджера по работе с клиентами.

Данные сохранены как customer_churn.csv. Вот поля и их определения:

    Name : Имя последнего контакта в компании
    Age: Возраст клиента
    Total_Purchase: Всего куплено рекламы
    Account_Manager: Бинарный 0=Нет менеджера, 1= Назначен менеджер по работе с клиентами
    Years: Общее количество лет в качестве клиента
    Num_sites: Количество веб-сайтов, которые используют услугу.
    Onboard_date: Дата, когда последний контакт был привлечен
    Location: Адрес головного офиса клиента
    Company: Название клиентской компании
    
После того как вы создали модель и оценили ее, проверьте модель на новых данных, которые сохранены под именем new_customers.csv. Необходимо узнать, какие клиенты наиболее вероятно уйдут (у них еще нет метки).

In [30]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic Regression').getOrCreate()
from pyspark.ml.classification import LogisticRegression

In [31]:
data = spark.read.csv('data/customer_churn.csv',inferSchema=True,
                     header=True)

In [32]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [33]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [34]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [35]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [36]:
# Индексация бинарного признака "Account_Manager"
account_manager_indexer = StringIndexer(inputCol="Account_Manager", outputCol="Account_Manager_Index")

In [37]:
# Индексация целевого признака "Churn"
churn_indexer = StringIndexer(inputCol="Churn", outputCol="label")

In [38]:
# Создаем экземпляр класса VectorAssembler
assembler = VectorAssembler(
    # Указываем входные колонки, которые будут использоваться для создания вектора признаков
    inputCols=["Age", "Total_Purchase", "Account_Manager_Index", "Years", "Num_Sites"],
    # Указываем имя выходной колонки, в которой будет храниться вектор признаков
    outputCol="features")

In [39]:
# Создаем экземпляр класса LogisticRegression
log_reg = LogisticRegression(featuresCol="features", labelCol="label")

In [40]:
from pyspark.ml import Pipeline

In [41]:
# Создание конвейера обработки данных и обучения модели
pipeline = Pipeline(stages=[account_manager_indexer, churn_indexer, assembler, log_reg])

In [42]:
# Разделяем данные на обучающий и тестовый наборы
# 70% данных идут в обучающий набор, 30% - в тестовый

train_data, test_data = data.randomSplit([0.7,0.3])

In [43]:
# Обучение модели
lrModel = pipeline.fit(train_data)

In [44]:
# Оценка модели на тестовых данных
results = lrModel.transform(test_data)
results.select("Churn", "prediction").show(10)

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [46]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
# Создаем экземпляр класса MulticlassClassificationEvaluator
# Столбец с прогнозами - 'prediction', столбец с истинными значениями - 'label'
# Метрика, которую мы хотим вычислить - 'accuracy' (точность)
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')
acc = evaluator.evaluate(results)
acc

0.8780487804878049

In [47]:
# Прогнозирование на новых данных
new_data = spark.read.csv('data/new_customers.csv', inferSchema=True, header=True)
new_data_transformed = lrModel.transform(new_data)
new_data_transformed.select("Company", "prediction").show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



## Задание №3. Модели основанные на решающих деревьях (Решающие деревья, Случайный лес, Градиентный бустинг)

- Для компании-произвозителя собачьего корма, необходимо попытаться предсказать, почему некоторые партии их корма портятся гораздо быстрее, чем предполагалось.

- К сожалению, эта компания по производству собачьего корма не обновила свое оборудование до последних моделей, что означает, что количество пяти консервантов, которые они используют, может значительно варьироваться. Но какой из консервантов оказывает наибольшее влияние?

- Сначала компания по производству собачьего корма готовит партию консерванта, которая содержит 4 разных консерванта (A, B, C, D), а затем добавляет "наполнитель". Ученые полагают, что один из консервантов A, B, C или D вызывает проблему и необходимо выяснить, какой именно.

Используйте методы машинного обучения, основанные на решающих деревьях, чтобы выяснить, какой признак имеет наибольшее значение, а затем выясните, какой химический элемент вызывает раннюю порчу.

* Pres_A : Процент консерванта A в смеси
* Pres_B : Процент консерванта B в смеси
* Pres_C : Процент консерванта C в смеси
* Pres_D : Процент консерванта D в смеси
* Spoiled: Метка, указывающая, испортилась ли партия собачьего корма.

___

**Тщательно подумайте о том, что на самом деле требуется решить в этой задаче!**
____

In [48]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rf').getOrCreate()

In [49]:
data = spark.read.csv('data/dog_food.csv',inferSchema=True,header=True)

In [50]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [51]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [52]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [53]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
# Создаем экземпляр класса VectorAssembler
assembler = VectorAssembler(
    # Указываем входные колонки, которые будут использоваться для создания вектора признаков
    inputCols=["A", "B", "C", "D"],
    # Указываем имя выходной колонки, в которой будет храниться вектор признаков
    outputCol="features")

In [54]:
data = assembler.transform(data)

In [55]:
# разделение на обучающую и тестовую
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [56]:
# модели:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
# Создание экземпляра классификатора случайного леса
rf = RandomForestClassifier(labelCol="Spoiled", featuresCol="features")

# Создание модели GBT
gbt = GBTClassifier(labelCol="Spoiled", featuresCol="features")

# Создание экземпляра DecisionTreeClassifier
dtc = DecisionTreeClassifier(labelCol='Spoiled',featuresCol='features')

In [57]:
# Обучение модели
rf_model = rf.fit(trainingData)
gbt_model = gbt.fit(trainingData)
dtc_model = dtc.fit(trainingData)

In [58]:
# Применение обученной модели для прогнозирования на тестовых данных.
rf_predictions = rf_model.transform(testData)
gbt_predictions = gbt_model.transform(testData)
dtc_predictions = dtc_model.transform(testData)

In [59]:
rf_predictions.select("prediction", "Spoiled", "features").show(5)

+----------+-------+------------------+
|prediction|Spoiled|          features|
+----------+-------+------------------+
|       1.0|    1.0|[1.0,1.0,12.0,2.0]|
|       1.0|    1.0|[1.0,1.0,13.0,3.0]|
|       0.0|    0.0| [1.0,4.0,9.0,3.0]|
|       0.0|    0.0| [1.0,5.0,8.0,3.0]|
|       0.0|    0.0| [1.0,7.0,8.0,4.0]|
+----------+-------+------------------+
only showing top 5 rows



In [60]:
gbt_predictions.select("prediction", "Spoiled", "features").show(5)

+----------+-------+------------------+
|prediction|Spoiled|          features|
+----------+-------+------------------+
|       1.0|    1.0|[1.0,1.0,12.0,2.0]|
|       1.0|    1.0|[1.0,1.0,13.0,3.0]|
|       0.0|    0.0| [1.0,4.0,9.0,3.0]|
|       0.0|    0.0| [1.0,5.0,8.0,3.0]|
|       0.0|    0.0| [1.0,7.0,8.0,4.0]|
+----------+-------+------------------+
only showing top 5 rows



In [61]:
dtc_predictions.select("prediction", "Spoiled", "features").show(5)

+----------+-------+------------------+
|prediction|Spoiled|          features|
+----------+-------+------------------+
|       1.0|    1.0|[1.0,1.0,12.0,2.0]|
|       1.0|    1.0|[1.0,1.0,13.0,3.0]|
|       0.0|    0.0| [1.0,4.0,9.0,3.0]|
|       0.0|    0.0| [1.0,5.0,8.0,3.0]|
|       0.0|    0.0| [1.0,7.0,8.0,4.0]|
+----------+-------+------------------+
only showing top 5 rows



In [62]:
# Оценка точности
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Spoiled", predictionCol="prediction", metricName="accuracy")

In [63]:
rf_accuracy = acc_evaluator.evaluate(rf_predictions)
gbt_accuracy = acc_evaluator.evaluate(gbt_predictions)
dtc_accuracy = acc_evaluator.evaluate(dtc_predictions)

In [64]:
print("Результаты:")
print('-'*80)
print('"Случайный лес" показал точность: {0:2.2f}%'.format(rf_accuracy*100))
print('-'*80)
print('Ансамбль деревьев с использованием GBT показал точность: {0:2.2f}%'.format(gbt_accuracy*100))
print('-'*80)
print('Одно решающее дерево показало точность: {0:2.2f}%'.format(dtc_accuracy*100))

Результаты:
--------------------------------------------------------------------------------
"Случайный лес" показал точность: 98.47%
--------------------------------------------------------------------------------
Ансамбль деревьев с использованием GBT показал точность: 95.42%
--------------------------------------------------------------------------------
Одно решающее дерево показало точность: 95.42%


In [65]:
rf_model.featureImportances

SparseVector(4, {0: 0.0208, 1: 0.029, 2: 0.9285, 3: 0.0217})

In [66]:
#  inputCols=["A", "B", "C", "D"],

# Задание №4. Кластеризация

- Крупная технологическая компания подверглась хакерской атаке. К счастью, их судебные инженеры смогли собрать ценные данные о взломах, включая информацию о продолжительности сессии, местоположении, скорости печати и т.д. Судебный инженер рассказывает, что удалось выяснить до сих пор, она смогла получить метаданные каждой сессии, которую хакеры использовали для подключения к их серверам. Вот особенности данных:
  * 'Session_Connection_Time': Сколько длилась сессия в минутах
  * 'Bytes Transferred': Количество МБ, переданных во время сессии
  * 'Kali_Trace_Used': Указывает, использовал ли хакер Kali Linux
  * 'Servers_Corrupted': Количество серверов, поврежденных во время атаки
  * 'Pages_Corrupted': Количество незаконно доступных страниц
  * 'Location': Место, откуда произошла атака (вероятно, бесполезно, потому что хакеры использовали VPN)
  * 'WPM_Typing_Speed': Их оценочная скорость печати на основе журналов сессий.

- У технологической компании есть 3 потенциальных хакера, которые совершили атаку. Они уверены в первых двух хакерах, но не очень уверены, был ли вовлечен третий хакер или нет.

- Вам необходимо выяснить, был ли третий подозреваемый как-то связан с атаками, или это были всего два хакера? Вероятно, наверняка это знать невозможно, но, возможно, то, что вы только что узнали о кластеризации, может помочь!

**Еще один ключевой факт, судебный инженер знает, что хакеры чередуют атаки. Это означает, что каждый из них должен иметь примерно одинаковое количество атак. Например, если было 100 общих атак, то в ситуации с двумя хакерами каждый должен был бы сделать около 50 взломов, в ситуации с тремя хакерами - каждый бы сделал около 33 взломов. Инженер считает, что это ключевой элемент для решения этой проблемы, но не знает, как различить эти неразмеченные данные на группы хакеров.**

In [67]:
spark = SparkSession.builder.appName("clustering").getOrCreate()

In [68]:
from pyspark.ml.clustering import KMeans

dataset = spark.read.csv("data/hack_data.csv",header=True,inferSchema=True)

In [69]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [70]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [71]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [72]:
dataset = dataset.drop("Location")

In [73]:
# Форматируем данные

In [74]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [75]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [76]:
final_data = vec_assembler.transform(dataset)

In [77]:
# Нормализация данных
from pyspark.ml.feature import StandardScaler

In [78]:
# Создание экземпляра StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [79]:
# Вычисление сводной статистики путем применения StandardScaler
scalerModel = scaler.fit(final_data)

In [80]:
# Нормализация каждого признака до единичного стандартного отклонения.
final_data = scalerModel.transform(final_data)

In [81]:
# Обучение модели k-means.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [82]:
# Оценка кластеризации путем вычисления суммы квадратов ошибок внутри групп.
wssse = model.summary.trainingCost
print("Сумма квадратов ошибок внутри групп = " + str(wssse))

Сумма квадратов ошибок внутри групп = 434.75507308487596


In [83]:
# Вывод результатов.
centers = model.clusterCenters()
print("Центры кластеров: ")
for center in centers:
    print(center)

Центры кластеров: 
[1.26023837 1.31829808 0.99280765 1.36491885 2.5625043  5.26676612]
[2.93719177 2.88492202 0.         3.19938371 4.52857793 3.30407351]
[3.05623261 2.95754486 1.99757683 3.2079628  4.49941976 3.26738378]


In [84]:
# Применение модели к данным и выбор столбца 'prediction'
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [85]:
predictions = model.transform(final_data)

In [86]:
predictions.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+



In [87]:
# Обучение модели k-means с 2 кластерами
kmeans_2 = KMeans(featuresCol='scaledFeatures',k=2)
model_2 = kmeans_2.fit(final_data)

In [88]:
predictions_2 = model_2.transform(final_data)

In [89]:
predictions_2.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



# Задание 5. Создание рекомендательной системы

- Необходимо построить систему рекомендаций для компании по доставке еды онлайн.

- Эта компания пытается выделиться на рынке, рекомендуя новые блюда клиентам на основе предпочтений других клиентов.

- **Ваш окончательный результат должен быть в виде функции, которая может принимать DataFrame Spark отдельного клиента с оценками различных блюд и выводить три лучших предложенных блюда.**

In [90]:
import pandas as pd

In [91]:
spark = SparkSession.builder.appName("RecommendationSystem").getOrCreate()

In [92]:
df = pd.read_csv('data/movielens_ratings.csv')

In [93]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [94]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [95]:
import numpy as np
df['mealskew'] = df['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [96]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


In [97]:
mealmap = { 2. : "Chicken Curry",
           3. : "Spicy Chicken Nuggest",
           5. : "Hamburger",
           9. : "Taco Surprise",
           11. : "Meatloaf",
           12. : "Ceaser Salad",
           15. : "BBQ Ribs",
           17. : "Sushi Plate",
           19. : "Cheesesteak Sandwhich",
           21. : "Lasagna",
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",
           27. : "Salmon with Mashed Potatoes",
           28. : "Penne Tomatoe Pasta",
           29. : "Pork Sliders",
           30. : "Vietnamese Sandwich",
           31. : "Chicken Wrap",
           np.nan: "Cowboy Burger",
           4. : "Pretzels and Cheese Plate",
           6. : "Spicy Pork Sliders",
           13. : "Mandarin Chicken PLate",
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",
           8. : "Chicken Chow Mein",
           10. : "Roasted Eggplant ",
           18. : "Pepperoni Pizza",
           22. : "Pulled Pork Plate",
           0. : "Cheese Pizza",
           1. : "Burrito",
           7. : "Nachos",
           24. : "Chili",
           20. : "Southwest Salad",
           25.: "Roast Beef Sandwich"}

In [98]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [99]:
df.to_csv('data/Meal_Info.csv',index=False)

In [100]:
data = spark.read.csv('data/Meal_Info.csv',inferSchema=True,header=True)

In [101]:
data.show()

+-------+------+------+--------+--------------------+
|movieId|rating|userId|mealskew|           meal_name|
+-------+------+------+--------+--------------------+
|      2|   3.0|     0|     2.0|       Chicken Curry|
|      3|   1.0|     0|     3.0|Spicy Chicken Nug...|
|      5|   2.0|     0|     5.0|           Hamburger|
|      9|   4.0|     0|     9.0|       Taco Surprise|
|     11|   1.0|     0|    11.0|            Meatloaf|
|     12|   2.0|     0|    12.0|        Ceaser Salad|
|     15|   1.0|     0|    15.0|            BBQ Ribs|
|     17|   1.0|     0|    17.0|         Sushi Plate|
|     19|   1.0|     0|    19.0|Cheesesteak Sandw...|
|     21|   1.0|     0|    21.0|             Lasagna|
|     23|   1.0|     0|    23.0|      Orange Chicken|
|     26|   3.0|     0|    26.0|    Spicy Beef Plate|
|     27|   1.0|     0|    27.0|Salmon with Mashe...|
|     28|   1.0|     0|    28.0| Penne Tomatoe Pasta|
|     29|   1.0|     0|    29.0|        Pork Sliders|
|     30|   1.0|     0|    3

________

In [102]:
(training, test) = data.randomSplit([0.8, 0.2])

In [103]:
from pyspark.ml.recommendation import ALS

In [104]:
# Создаем экземпляр ALS (Alternating Least Squares - метод чередующихся наименьших квадратов)
# maxIter=5 - максимальное количество итераций
# regParam=0.01 - параметр регуляризации
# userCol="userId" - колонка с идентификаторами пользователей
# itemCol="movieId" - колонка с идентификаторами фильмов
# ratingCol="rating" - колонка с рейтингами
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

In [105]:
# Обучаем модель на тренировочных данных
model = als.fit(training)

In [106]:
predictions = model.transform(test)

In [107]:
predictions.show()

+-------+------+------+--------+--------------------+------------+
|movieId|rating|userId|mealskew|           meal_name|  prediction|
+-------+------+------+--------+--------------------+------------+
|      2|   1.0|    26|     2.0|       Chicken Curry|  0.87328017|
|      4|   1.0|    12|     4.0|Pretzels and Chee...|  -1.2910495|
|      0|   1.0|    22|     0.0|        Cheese Pizza|   2.9311786|
|      6|   1.0|     1|     6.0|  Spicy Pork Sliders|-0.073086284|
|      5|   1.0|    13|     5.0|           Hamburger|   1.6010289|
|      6|   1.0|    20|     6.0|  Spicy Pork Sliders|   1.4649295|
|      1|   1.0|     5|     1.0|             Burrito|  -1.8046918|
|      8|   1.0|     5|     8.0|   Chicken Chow Mein|   2.2998126|
|      0|   1.0|    19|     0.0|        Cheese Pizza|   0.7118062|
|      2|   3.0|     9|     2.0|       Chicken Curry| -0.33303112|
|      4|   1.0|     9|     4.0|Pretzels and Chee...|   1.3780953|
|      2|   4.0|     8|     2.0|       Chicken Curry|   1.9512

In [108]:
from pyspark.ml.evaluation import RegressionEvaluator

In [109]:
# Создаем экземпляр RegressionEvaluator для оценки модели
# metricName="rmse" - используем метрику RMSE (Root Mean Square Error - квадратный корень из среднего квадрата ошибки)
# labelCol="rating" - колонка с истинными значениями рейтинга
# predictionCol="prediction" - колонка с предсказанными значениями рейтинга
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

In [110]:
# Вычисляем значение RMSE на предсказаниях модели
rmse = evaluator.evaluate(predictions)

# Выводим значение RMSE
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.750837018137974


In [111]:
# Предложим рекомендации пользователю
# Функция
def recommend_for_user(user_df, num_recommend=3):
    recommend = model.transform(user_df)
    recommend = recommend.orderBy('prediction', ascending=False)
    top_recommend = recommend.limit(num_recommend)
    # Присоединяем информацию о блюдах и удаляем дубликаты
    top_recommend = top_recommend.join(data, on='movieId', how='inner') \
                                 .select('movieId', 'meal_name', 'prediction') \
                                 .distinct()

    return top_recommend

In [112]:
# Проверка
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])
recommend = recommend_for_user(single_user)
recommend.show()

+-------+-------------+----------+
|movieId|    meal_name|prediction|
+-------+-------------+----------+
|     88|Cowboy Burger| 3.4117208|
|     36|Cowboy Burger| 4.2782655|
|     80|Cowboy Burger|   4.96745|
+-------+-------------+----------+

